In [2]:
from libs import *

In [3]:
from pymittagleffler import *

In [198]:
class Solution:
    def __init__(self, alpha, lam, k):
        self.al = alpha
        self.lam = lam
        self.k = k
    def mitlef (self, al, bt, t):
        # t is np.ndarray  
        return np.real(mittag_leffler(t, al, bt))
    def val(self, points):
        t = points[..., 0:1] #N*1 
        x = points[..., 1:2] #N*1
        part1 = np.sin(self.k*np.pi*x) #N*1 
        part2 = self.mitlef(self.al, 1.0, -self.lam*np.power(t,self.al)) - 0.5*t*self.mitlef(self.al, 2.0, -self.lam*np.power(t,self.al)) #N*1
        return part1*part2 #N*1
    def dt(self, points):
        t = points[..., 0:1] #N*1 
        x = points[..., 1:2] #N*1
        part1 = np.sin(self.k*np.pi*x) #N*1 
        part2 = (-self.lam)*np.power(t,self.al-1)*self.mitlef(self.al, self.al, -self.lam*np.power(t,self.al)) - 0.5*self.mitlef(self.al, 1, -self.lam*np.power(t,self.al)) #N*1
        return part1*part2 #N*1
    def dtal(self, points, method, nums):
        if method=='MC-I':
            lens = len(points)
            al=self.al
            eps = 1e-10
            coeff = sp.gamma(2-al)
            taus = beta.rvs(2-al,1,size=nums)
            dt = self.dt(points)
            # compute u'(0,x)
            t0 = np.concatenate((np.zeros_like(points[:, 0:1]), points[:,1:]), axis=1)
            dt0 = self.dt(t0)
            # compute u'(t-t tau,x) 
            t = points[:, 0:1] #N*1
            taus = taus.reshape(-1,1) #M*1
            x = points[:, 1:2] #N*1
            t_tau = t.T * taus.reshape(-1,1) #M*N 
            t_t_tau = points[:, 0]-t_tau #M*N 
            t_tau_max = np.maximum(t_tau, eps) #M*N
            new_x = np.tile(x.reshape(1,-1), (nums,1))#M*N
            new_points = np.stack((t_t_tau,new_x), axis=2) #M*N*2 
            dttau = self.dt(new_points).reshape(nums,lens, 1) #M*N*1
            part1 = np.mean((dt-dttau)/t_tau_max.reshape(nums,lens,1), axis=0)# N*1
            part1 = part1*(al-1.0)/(2.0-al)*np.power(points[:, 0:1], 2-al) 
            part2 = (dt-dt0)*np.power(points[:, 0:1],1-al) 
            dtal = (part1+part2)/coeff #N*1 
            return dtal
        if method=='GJ-I':
            lens = len(points)
            quad_t, quad_wt = roots_jacobi(nums, 0, 1-self.al)
            self.quad_t = (quad_t + 1) / 2
            self.quad_w = quad_wt * (1 / 2) ** (2-self.al)
            al=self.al
            coeff = sp.gamma(2-al)
            taus = self.quad_t 
            quad_w = self.quad_w.reshape(-1,1,1)#M*1*1
            # quad_w = np.tile(quad_w, (1,lens,1)) #M*N*1
            dt = self.dt(points)
            # compute u'(0,x)
            t0 = np.concatenate((np.zeros_like(points[:, 0:1]), points[:,1:]), axis=1)
            dt0 = self.dt(t0)
            # compute u'(t-t tau,x) 
            t = points[:, 0:1] #N*1
            taus = taus.reshape(-1,1) #M*1
            x = points[:, 1:2] #N*1
            t_tau = t.T * taus.reshape(-1,1) #M*N 
            t_t_tau = points[:, 0]-t_tau #M*N 
            new_x = np.tile(x.reshape(1,-1), (nums,1))#M*N
            new_points = np.stack((t_t_tau,new_x), axis=2) #M*N*2 
            dttau = self.dt(new_points).reshape(nums,lens, 1) #M*N*1
            part1 =  np.sum(quad_w*(dt-dttau)/t_tau.reshape(nums,lens,1), axis=0)
            part1 = part1*(al-1.0)*np.power(points[:, 0:1], 2-al) 
            part2 = (dt-dt0)*np.power(points[:, 0:1],1-al) 
            dtal = (part1+part2)/coeff #N*1 
            return dtal

In [27]:
tlim =(0,2)
xlim=[[0,1]]
dev = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [76]:
batch = {'in':3,'bd':10,'init':10}

In [77]:
sampler = iter(TimeSpaceEasySampler(axeslim=xlim, tlim=tlim, dev=dev, batch=batch))

In [78]:
points = sampler.sample()
points = points['in'].detach().cpu().numpy()

In [203]:
u = Solution(alpha=1.25, lam=1, k=1)

In [204]:
dtal_true = -u.lam*u.val(points)

In [147]:
dtal1 = u.dtal(points, method='MC-I', nums=1000)

In [205]:
dtal2 = u.dtal(points,method='GJ-I', nums=10)

In [206]:
(dtal2-dtal_true)/dtal_true

array([[-7.95900285e-04],
       [-6.67386747e-05],
       [ 2.30239298e-04]])